In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
import os
import pickle
import glob
import json
import warnings
import pandas as pd
import numpy as np
import collections
from datetime import datetime
import getpass
from pathlib import Path

from helper_eval import evalAssays, evalClinvar, evalUKBB, evalDDD

aaPosCols = ["gene_name", "change_position_1based", "ref_aa", "alt_aa"]
dnaPosCols = ['chr', 'pos', 'non_flipped_ref', 'non_flipped_alt']

In [4]:
datafolder = <PATH_TO_DATA>
#datafolder = "/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package"

In [6]:
precomputedScoreFilePaths = glob.glob( datafolder + "/pai_scores/*" )

scoreDF = None
for scoreFilePath in precomputedScoreFilePaths:
    print(scoreFilePath)
    
    df = pd.read_csv(scoreFilePath)
    df = df[ aaPosCols + [ col for col in df.columns if col.startswith("score") ] ].drop_duplicates(subset=aaPosCols)

    if scoreDF is None:
        scoreDF = df
    else:
        scoreDF = scoreDF.merge(df, on=aaPosCols)

/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/ensemble_scores/pai3d.ensPrimAndHum.csv
/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/ensemble_scores/pai3d.ensPrim.orig.csv
/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/ensemble_scores/pai3d.ensHum.csv


In [7]:
singleModelScoreFilePaths = glob.glob( datafolder + "/pai_trainAndEvalData/human_singlemodels/scores/*" )

for i, scoreFile in enumerate(singleModelScoreFilePaths):
    print(scoreFile)
    
    df_local = pd.read_csv(scoreFile)[aaPosCols + ["score"]].drop_duplicates( subset=aaPosCols )
    df_local["score"] = df_local["score"].rank() / len(df_local)
    df_local.rename(columns = {"score": "score_human_%d" % i}, inplace=True)
    
    scoreDF = scoreDF.merge(df_local, on=aaPosCols)
    
    

/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/human_singlemodels/scores/3.wholeProteomePreds.csv.gz
/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/human_singlemodels/scores/8.wholeProteomePreds.csv.gz
/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/human_singlemodels/scores/9.wholeProteomePreds.csv.gz
/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/human_singlemodels/scores/7.wholeProteomePreds.csv.gz
/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/human_singlemodels/scores/4.wholeProteomePreds.csv.gz
/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/human_singlemodels/scores/0.wholeProteomePreds.csv.gz
/illumina/scratch/deep_learning/thamp/PAI/2025_06_12_oldPaiCodeRelease/data/package/ens/human_singlemodels/scores/6.wholeProteomePreds.csv.gz
/illum

In [8]:
scoreDF.to_pickle("/tmp/scoresDF.pkl")

In [ ]:
# scoreCols = [ col for col in scoreDF.columns if "score_human_" in col ]
# scoreDF["score_PAI3D_human"] = scoreDF[ scoreCols ].mean(axis=1)
# scoreCols = [ col for col in scoreDF.columns if col.startswith("score") ]

In [12]:
scoreCols = [ col for col in scoreDF.columns if col.startswith("score") ]

In [13]:
scoreDF.head()

,gene_name,change_position_1based,ref_aa,alt_aa,score_PAI3D_primatehuman,score_PAI3D,score_PAI3D_human,score_human_0,score_human_1,score_human_2,score_human_3,score_human_4,score_human_5,score_human_6,score_human_7,score_human_8,score_human_9
0,ENST00000000233.10,2,G,A,0.544681,0.646401,0.417073,0.494361,0.440581,0.430280,0.400248,0.410005,0.406358,0.372263,0.436566,0.369509,0.410557
1,ENST00000000233.10,2,G,C,0.774804,0.789498,0.194336,0.208375,0.199393,0.306178,0.164666,0.170791,0.172557,0.231877,0.148179,0.124866,0.216482
2,ENST00000000233.10,2,G,D,0.643732,0.710977,0.325057,0.315775,0.379711,0.431314,0.281847,0.383750,0.368678,0.309183,0.281050,0.223586,0.275672
3,ENST00000000233.10,2,G,R,0.660441,0.730334,0.321817,0.347880,0.276137,0.349453,0.323394,0.344386,0.351190,0.338260,0.343363,0.239167,0.304936
4,ENST00000000233.10,2,G,S,0.510143,0.614245,0.435551,0.452561,0.452572,0.430248,0.455984,0.452988,0.527283,0.374647,0.484561,0.375744,0.348921


In [14]:
evalNameDict = {"dddAsdChd_2023": [datafolder + "/pai_trainAndEvalData/evaluation/dddAsdChd_2023.csv", evalDDD],
                "clinvar2023": [datafolder + "/pai_trainAndEvalData/evaluation/clinvar2023.csv", evalClinvar],
                "assays2023_02": [datafolder + "/pai_trainAndEvalData/evaluation/assays2023_02.csv", evalAssays],
                "bb2023_01": [datafolder + "/pai_trainAndEvalData/evaluation/bb2023_01.csv", evalUKBB]}

evalDFs = []

for evalName, (evalFilePath, evalFun) in evalNameDict.items():
    print(evalName)
    evalDF = evalFun(        evalFilePath,
                             scoreDF,
                             scoreCols)
    evalDFs.append(evalDF)

evalDF = pd.concat(evalDFs)

dddAsdChd_2023
clinvar2023
assays2023_02
bb2023_01


In [15]:
evalDF["statName"] = evalDF["dataset"] + "_" + evalDF["statName"]
evalDF.drop(columns=["dataset"], inplace=True)
evalDF = evalDF.set_index(['scoreName', "statName"])['stat_mean'].unstack(fill_value=0).reset_index()
evalDF.columns.name = "index"
evalDF

index,scoreName,DDD_asd_control_log10pval,DDD_chd_control_log10pval,DDD_ddd_control_log10pval,UKBB_corr,assay_corr,clinvar_auc
0,score_PAI3D,7.767343,3.999850,66.793698,0.283362,0.421079,0.927198
1,score_PAI3D_human,8.248613,5.122499,67.357802,0.283882,0.423294,0.923793
2,score_PAI3D_primatehuman,8.101957,4.590955,68.044436,0.285944,0.426835,0.926541
3,score_human_0,7.779054,5.032246,65.844366,0.280526,0.420203,0.922226
4,score_human_1,8.201949,4.516603,67.010679,0.278169,0.416766,0.921917
5,score_human_2,8.650900,5.245379,68.440812,0.275726,0.412340,0.918420
6,score_human_3,7.779070,5.033207,64.506292,0.281789,0.422717,0.922716
7,score_human_4,8.036804,5.108710,65.094185,0.280558,0.420915,0.919337
8,score_human_5,8.114431,5.160445,65.147640,0.281388,0.420118,0.921689
9,score_human_6,8.040737,4.910156,67.719937,0.279305,0.407212,0.919456
